<a href="https://colab.research.google.com/github/MMathisLab/DeepLabCut/blob/master/examples/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590444465547-SHXODUII311HEE407IL6/ke17ZwdGBToddI8pDm48kE4VnnB9_j2k1VP236ADqAFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxQg9Vf0owGyf3dhfDKy8SxMujaKmp2B54Sb3VS1rO76Whq-cUhHVuKFlGUXsU9tJk/ezgif.com-video-to-gif.gif?format=1500w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub.

Mathis et al, in prep. <- please note, we are providing this toolbox as an early access release; more feeatures and details will be released with the forthcoming paper.




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#GUIs don't work on the cloud, so we supress them:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "5"
#os.environ["DLClight"]="True"
import deeplabcut
import tensorflow as tf

2022-10-03 15:35:37.970994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 15:35:38.239022: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 15:35:38.239040: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-03 15:35:38.275783: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 15:35:39.203615: W tensorflow/stream_executor/platform/de

Loading DLC 2.2.2...


'\nvers = (tf.__version__).split(".")\nif int(vers[0]) == 1 and int(vers[1]) > 12:\n    TF = tf.compat.v1  # behaves differently before 1.13\nelse:\n    TF = tf\n\nTF.logging.set_verbosity(TF.logging.ERROR)\nDEBUG = True and "DEBUG" in os.environ and os.environ["DEBUG"]\nfrom deeplabcut import DEBUG\n'

In [ ]:
#let's make sure we see a GPU:
tf.test.gpu_device_name()

## YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [2]:
# PLEASE EDIT THIS:
ProjectFolderName = 'Cricket20-Yuchen-2022-08-22'
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path


#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/config.yaml'
path_config_file
#This creates a path variable that links to your google drive copy

'/home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/config.yaml'

## Create a multi-animal training dataset:
### You must do this step inside of Colab:

- Reminder: you must connect EVERY bodypart in a skeleton before you run this step! See docs for crucial details on how to do this effciently: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#b-configure-the-project-

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589256735280-SCN7CROSJNJWCDS6EK5T/ke17ZwdGBToddI8pDm48kB08p9-rNkpPD7A3fw8YFjZZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZamWLI2zvYWH8K3-s_4yszcp2ryTI0HqTOaaUohrI8PIno0kSvzOWihTW1zp8-1-7mzYxUQjsVr2n3nmNdVcso4/bodyparts-skeleton.png?format=1000w)
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589410182515-9SJO9MML6CNCXBAWQ6Z6/ke17ZwdGBToddI8pDm48kJ1oJoOIxBAgRD2ClXVCmKFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxBw7VlGKDQO2xTcc51Yv6DahHgScLwHgvMZoEtbzk_9vMJY_JknNFgVzVQ2g0FD_s/ezgif.com-video-to-gif+%2811%29.gif?format=750w)

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
#deeplabcut.cropimagesandlabels(path_config_file, size=(400, 400), userfeedback=False)
deeplabcut.add_new_videos(path_config_file, ['/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0637.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0638.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0639.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0640.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0641.mp4',
                                             '/home/yuchen/sftpFolder/DeepLabCut/'+ProjectFolderName+'/videos/PIC_0642.mp4'], copy_videos=False)
#if you labeled on Windows, please set the windows2linux=True:

In [ ]:
deeplabcut.extract_frames(path_config_file, mode='automatic', algo='kmeans', userfeedback=False, crop=True)

In [4]:
deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


In [ ]:
deeplabcut.check_labels(path_config_file)

In [ ]:
deeplabcut.SkeletonBuilder(path_config_file)

In [3]:
deeplabcut.refine_labels(path_config_file)

Gtk-Message: 15:36:04.427: Failed to load module "canberra-gtk-module"


Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!


/home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/deeplabcut/gui/refinement.py:736: FutureWarning: inplace is deprecated and will be removed in a future version.
  [self.scorer.replace(self.scorer, self.humanscorer)], level=0, inplace=True


Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [ ]:
deeplabcut.merge_datasets(path_config_file)

In [4]:
deeplabcut.create_training_dataset(path_config_file, augmenter_type='imgaug', windows2linux=True)
#deeplabcut.create_multianimaltraining_dataset(path_config_file, windows2linux=False)

/home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/deeplabcut/generate_training_dataset/trainingsetmanipulation.py:823: FutureWarning: `windows2linux` has no effect since 2.2.0.4 and will be removed in 2.2.1.
  FutureWarning,


The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([106,   7, 125, 160, 136,  71,  56, 151, 162, 108,  33, 153, 158,
            5,  18,  92, 164, 143,  37, 101, 144,  60,  63,   4, 122, 165,
          113, 166, 168,  61,  44,  26, 107, 131,  66,   8,  55,  83,  45,
          163, 123,  24,  30,  95,  54,  80,  19, 137, 104, 146,  16,  51,
          170, 121,  40,  74,  22, 110, 111, 129,  96,  90, 116,  27,  94,
          157, 130, 176,  89,  62,   2,  59,  97,  98,  43,  10,  86,  73,
          173, 161, 112, 138,  93, 150, 179,  50, 177, 118,  64, 124, 126,
           69,  49,  48,  85,  13, 135,  23, 167,  20,  15,  78,  52, 100,
           76,   3, 178, 109,   6,  68,  75,  84, 134,  12, 139, 154,  14,
            0,  91, 155,  46,  11, 119, 102,  35,  57,  41, 175,  65,   1,
          120, 145,  42, 105, 132, 156,  17,  38, 133,  53, 141, 128,  34,
           28, 114,  31, 149, 127, 159,  32, 142, 152, 147,  29,  99,  82,
           79, 115, 148, 174,  72,  77,  25,  81, 169, 171,  39,  58, 140,
           8

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [5]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. 
#Typically, you want to train to 50,000 iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

#which shuffle do you want to train?
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=1000, saveiters=1000, maxiters=300000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24]],
 'all_joints_names': ['Head',
                      'Pro',
                      'Meso',
                      'Meta',
                      'LF0',
                      'LF1',
                      'LF2',
                      'LM0',
                      'LM1',
                      'LM2',
                      'LH0',
                      'LH1',
                      'LH2',
                      'RF0',
                      'RF1',
             

Selecting single-animal trainer
Batch Size is 1
Loading ImageNet-pretrained resnet_50


2022-09-19 12:56:54.863162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA RTX A6000 major: 8 minor: 6 memoryClockRate(GHz): 1.8
pciBusID: 0000:a1:00.0
2022-09-19 12:56:54.863381: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/cv2/../../lib64:/opt/slurm/20.02.6/lib
2022-09-19 12:56:54.863444: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/cv2/../../lib64:/opt/slurm/20.02.6/lib
2022-09-19 12:56:54.863506: W tensorflow/stream_executor/platform/default/dso_loader.cc:55

Max_iters overwritten as 300000
Display_iters overwritten as 1000
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/dlc-models/iteration-0/CricketAug22-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3], [

iteration: 1000 loss: 0.0354 lr: 0.005
iteration: 2000 loss: 0.0204 lr: 0.005
iteration: 3000 loss: 0.0167 lr: 0.005
iteration: 4000 loss: 0.0145 lr: 0.005
iteration: 5000 loss: 0.0135 lr: 0.005
iteration: 6000 loss: 0.0122 lr: 0.005
iteration: 7000 loss: 0.0114 lr: 0.005
iteration: 8000 loss: 0.0109 lr: 0.005
iteration: 9000 loss: 0.0104 lr: 0.005
iteration: 10000 loss: 0.0099 lr: 0.005
iteration: 11000 loss: 0.0110 lr: 0.02
iteration: 12000 loss: 0.0099 lr: 0.02
iteration: 13000 loss: 0.0088 lr: 0.02
iteration: 14000 loss: 0.0082 lr: 0.02
iteration: 15000 loss: 0.0076 lr: 0.02
iteration: 16000 loss: 0.0074 lr: 0.02
iteration: 17000 loss: 0.0070 lr: 0.02
iteration: 18000 loss: 0.0066 lr: 0.02
iteration: 19000 loss: 0.0065 lr: 0.02
iteration: 20000 loss: 0.0063 lr: 0.02
iteration: 21000 loss: 0.0062 lr: 0.02
iteration: 22000 loss: 0.0060 lr: 0.02
iteration: 23000 loss: 0.0059 lr: 0.02
iteration: 24000 loss: 0.0057 lr: 0.02
iteration: 25000 loss: 0.0056 lr: 0.02
iteration: 26000 loss: 0

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


2022-09-21 11:51:36.232555: W tensorflow/core/kernels/queue_base.cc:277] _0_fifo_queue: Skipping cancelled enqueue attempt with queue not closed
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1365, in _do_call
    return fn(*args)
  File "/home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1350, in _run_fn
    target_list, run_metadata)
  File "/home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1443, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/threading.p

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: for maDLC, this is several steps. 
 - First, we evaluate the pose estimation performance, and then we can cross-valudate optimal inference parameters.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [6]:
#%matplotlib notebook
#let's evaluate first:
%matplotlib inline
deeplabcut.evaluate_network(path_config_file,plotting=False)
#plot a few scoremaps:
#deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])


#and begin to cross-validate:
#deeplabcut.evaluate_multianimal_crossvalidate(
#            path_config_file,
#            Shuffles=[shuffle],
#            edgewisecondition=True,
#            leastbpts=1,
#            init_points=20,
#            n_iter=100,
#            target='rpck_train',
#        )

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24]],
 'all_joints_names': ['Head',
                      'Pro',
                      'Meso',
                      'Meta',
                      'LF0',
                      'LF1',
                      'LF2',
                      'LM0',
                      'LM1',
                      'LM2',
                      'LH0',
                      'LH1',
                      'LH2',
                      'RF0',
                      'RF1',
             

Running  DLC_resnet50_CricketAug22shuffle1_300000  with # of training iterations: 300000


2022-09-21 11:51:37.750416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA RTX A6000 major: 8 minor: 6 memoryClockRate(GHz): 1.8
pciBusID: 0000:a1:00.0
2022-09-21 11:51:37.750642: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/cv2/../../lib64:/opt/slurm/20.02.6/lib
2022-09-21 11:51:37.750701: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yuchen/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/cv2/../../lib64:/opt/slurm/20.02.6/lib
2022-09-21 11:51:37.750757: W tensorflow/stream_executor/platform/default/dso_loader.cc:55

Running evaluation ...


180it [00:32,  5.62it/s]


Analysis is done and the results are stored (see evaluation-results) for snapshot:  snapshot-300000
Results for 300000  training iterations: 95 1 train error: 2.0 pixels. Test error: 3.07  pixels.
With pcutoff of 0.4  train error: 2.0 pixels. Test error: 3.07 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
Please check the results, then choose the best model (snapshot) for prediction. You can update the config.yaml file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise, consider adding more labeled-data and retraining the network (see DeepLabCut workflow Fig 2, Nath 2019)


^ NOTE: this optimized part detection for video analysis. It cannot optimze for tracking, as this is use-case dependent. Please check the docs on how you can set the best parameters and modify/test before "final" tracking parameters. You can use COLAB to analyze videos, but afterwards we recommend using the outputs/proejct folder locally to run the final steps! They do not require a GPU. 

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [7]:
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype='.mp4', save_as_csv=True)
deeplabcut.create_labeled_video(path_config_file,videofile_path,videotype='.mp4')
deeplabcut.analyzeskeleton(path_config_file, videofile_path, videotype='.mp4', shuffle=1, save_as_csv=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24]],
 'all_joints_names': ['Head',
                      'Pro',
                      'Meso',
                      'Meta',
                      'LF0',
                      'LF1',
                      'LF2',
                      'LM0',
                      'LM1',
                      'LM2',
                      'LH0',
                      'LH1',
                      'LH2',
                      'RF0',
                      'RF1',
             

Using snapshot-300000 for model /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/dlc-models/iteration-0/CricketAug22-trainset95shuffle1


2022-09-21 11:56:32.852571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-09-21 11:56:32.852635: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      


Analyzing all the videos in the directory...
Starting to analyze %  /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0639.mp4
Loading  /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0639.mp4
Duration of video [s]:  130.63 , recorded with  59.94 fps!
Overall # of frames:  7830  found with (before cropping) frame dimensions:  640 360
Starting to extract posture


100%|█████████▉| 7800/7830 [15:02<00:03,  8.64it/s]


...

Generating frames and creating video.Overall # of frames: 7815 with cropped frame dimensions: 640 360


Generating frames and creating video.Overall # of frames: 7830 with cropped frame dimensions: 640 360


  0%|          | 0/7830 [00:00<?, ?it/s]


Generating frames and creating video.


100%|██████████| 7830/7830 [00:32<00:00, 242.41it/s]


Analyzing all the videos in the directory...
Processing /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0638.mp4
Processing /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0641.mp4
Processing /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0640.mp4
Processing /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0642.mp4
Processing /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0639.mp4
Processing /home/yuchen/sftpFolder/DeepLabCut/Cricket20-Yuchen-2022-08-22/videos/PIC_0637.mp4


In [ ]:
deeplabcut.extract_outlier_frames(path_config_file, videofile_path)